In [ ]:
import numpy as np

import adaptive_latents
from adaptive_latents import AnimationManager, Pipeline, CenteringTransformer, proSVD, Bubblewrap, VanillaOnlineRegressor, NumpyTimedDataSource, KernelSmoother, Concatenator, sjPCA
import adaptive_latents.input_sources.datasets as datasets
import adaptive_latents.plotting_functions as pf
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from IPython.display import Video, Image, display


In [ ]:
!hostname


## Combination and prediction video


In [ ]:
%%time
%matplotlib inline
d = datasets.Odoherty21Dataset()

streams = []
streams.append( (d.neural_data,     0) )
streams.append( (d.behavioral_data, 1) )
streams.append( (d.behavioral_data, 3) )

p1 = Pipeline([
    CenteringTransformer(input_streams={0:'X'}, output_streams={0:0}),
    KernelSmoother(tau=0.08/d.bin_width, input_streams={0:'X'}, output_streams={0:0}),
    Concatenator(input_streams={0: 0, 1: 1}, output_streams={0:2, 1:2, 'skip':-1}),
    pro:=proSVD(k=6, init_size=100, input_streams={2:'X'}, output_streams={2:2}),
    sjPCA(init_size=100, input_streams={2:'X'}, output_streams={2:2}),
])

p2 = Pipeline([
    bw:=Bubblewrap(
        num=500,
        eps=1e-6,
        step=1e-3,
        num_grad_q=1,
        input_streams={2:'X'},
        output_streams={2:2},
        log_level=1,
    ),
    reg:=VanillaOnlineRegressor(
        input_streams={2:'X', 3:'Y'},
        output_streams={2:2},
    ),
])



video_dt = 0.1
video_ts = np.arange(50)*video_dt + 25
streams.append( (NumpyTimedDataSource(np.nan * video_ts, video_ts), 'video') )


pbar = tqdm(total=video_ts[-1])
latents = []
latent_ts = []

latent_predictions = []
beh_predictions = []
prediction_ts = []

pred_am = None
bw_am = None
with (
    plt.ioff(),
    AnimationManager(fps=1/video_dt, filetype='mp4') as bw_am, 
    (vid:=pf.PredictionVideo(d, fps=1/video_dt, filetype='mp4')).am as pred_am
):
    for output, stream in Pipeline().streaming_run_on(streams, return_output_stream=True):
        # dim reduction part of pipeline
        output, stream = p1.partial_fit_transform(output, stream, return_output_stream=True)
        if stream == 2 and np.isfinite(output).all():
            latents.append(output)
            latent_ts.append(output.t)

        # prediction part of pipeline
        output, stream = p2.partial_fit_transform(output, stream, return_output_stream=True)
        if stream == 2 and np.isfinite(output).all():
            prediction_t = output.t+bw.dt*1
            alpha_pred = bw.get_alpha_at_t(prediction_t)
            latent_prediction = np.array(bw.mu[np.argmax(alpha_pred)])
            beh_prediction = reg.predict(alpha_pred)

            latent_predictions.append(latent_prediction)
            beh_predictions.append(beh_prediction)
            prediction_ts.append(prediction_t)
            
        
        if stream == 'video':
            
            if bw_am is not None:
                if output.t == video_ts[0]:
                    ax: plt.Axes = bw_am.axs[0,0]
                    ax.text(0,1, f"p1={str(p1)}\np2={str(p2)}\nmin video t={video_ts.min()}", transform=ax.transAxes, fontsize='xx-small', verticalalignment='top', wrap=True)
                    bw_am.grab_frame()
                    
                ax: plt.Axes = bw_am.axs[0,0]
                ax.cla()
                bw.scatter_data_with_decay(ax,np.squeeze(latents))
                bw.show_active_bubbles_and_connections_2d(ax, np.squeeze(latents))
                bw_am.grab_frame()
                
            if pred_am is not None:
                if output.t == video_ts[0]:
                    ax = vid.neural_data_ax
                    ax.text(0,1, f"p1={str(p1)}\np2={str(p2)}\nmin video t={video_ts.min()}", transform=ax.transAxes, fontsize='xx-small', verticalalignment='top', wrap=True)
                    ax.set_zorder(1)
                    pred_am.grab_frame()
                
                vid.plot_for_video_t(output.t, latents, latent_ts, latent_predictions, beh_predictions, prediction_ts)
            
            if output.t == video_ts[-1]:
                break

        pbar.update(output.t - pbar.n)


latents = np.squeeze(latents)
latent_predictions = np.squeeze(latent_predictions)
beh_predictions = np.squeeze(beh_predictions)
prediction_ts = np.squeeze(prediction_ts)

plt.close()
plt.close()
Bubblewrap.compare_runs([bw]], t_in_samples=True)
plt.show()

if pred_am is not None:
    display(Video(pred_am.outfile, embed=True, width=900))


if bw_am is not None:
    display(Video(bw_am.outfile, embed=True, width=700))

fig, ax = plt.subplots()
ax.matshow(bw.log['alpha'][-30:])
ax.set_title('alpha')


In [ ]:

test_s = (
    (d.behavioral_data.t > ((prediction_ts[0] + prediction_ts[1])/2))
    &
    (np.isfinite(d.behavioral_data.a).all(axis=2)[:,0])
)

# estimates = adaptive_latents.utils.resample_matched_timeseries(beh_predictions, prediction_ts, d.behavioral_data.t[test_s])
# targets = d.behavioral_data.a[test_s,0,:]

estimates = adaptive_latents.utils.resample_matched_timeseries(d.behavioral_data.a[test_s,0,:],d.behavioral_data.t[test_s],  prediction_ts)
targets = beh_predictions

errors = estimates - targets

print([np.corrcoef(estimates[:,i], targets[:,i])[0,1] for i in range(3)])

In [ ]:
for i in range(3):
    plt.plot(prediction_ts/bw.dt, estimates[:,i], color=f'C{i}')
    plt.plot(prediction_ts/bw.dt, targets[:,i], color=f'C{i}', alpha=.5, label=f'{np.corrcoef(estimates[:,i], targets[:,i])[0,1]:.3f}')
plt.xlabel("time (samples)")
plt.ylabel("behavior")
plt.legend()

In [ ]:

%matplotlib inline
fig, ax = plt.subplots()
s = np.array(latent_ts) > 10
# ax.scatter(latents[s,0], latents[s,1], s=5, alpha=.1, c='C0', edgecolors='none')
sub_latents = latents[s]
d_sub_latents = np.diff(sub_latents, axis=0)
d_sub_latents = d_sub_latents / np.linalg.norm(d_sub_latents, axis=1)[:, np.newaxis]

ax.quiver(sub_latents[:-1,0], sub_latents[:-1,1], d_sub_latents[:,0], d_sub_latents[:,1], scale=1/20, units='dots')

x1, x2, y1, y2 = ax.axis()

x_points = np.linspace(x1, x2, 10)
y_points = np.linspace(y1, y2, 10)
mgrid = np.meshgrid(*[np.linspace(sub_latents[:,i].min(),sub_latents[:,i].max(), 10) for i in range(sub_latents.shape[1])])

# arrows = np.zeros(shape=(len(points), sub_latents.shape[1]))

# ax.quiver(points[:,0], points[:,1])


In [ ]:

fig, ax = plt.subplots()

pf.plot_history_with_tail(ax, latents)
means = np.mean(latents, axis=0)
ax.arrow(latents[-1,0], latents[-1,1], means[0] - latents[-1,0], means[1] - latents[-1,1], zorder=5, head_width=.1, color='k')

ax.axis('off');



In [ ]:
assert isinstance(d, datasets.Zong22Dataset)
desired_pattern = np.mean(latents[-101:-1], axis=0)
current_pattern = latents[-1]
desired_stim = desired_pattern - current_pattern

desired_pattern = (desired_stim @ pro.Q.T)[:d.neural_data.a.shape[-1]]

near_zero = np.abs(desired_stim) < 100
print(near_zero.sum())
desired_stim[near_zero] = np.nan


fig, ax = plt.subplots()
d.show_stim_pattern(ax, np.abs(desired_pattern))


## Intro video

In [ ]:
d = datasets.Odoherty21Dataset()

tail_length = 5  # seconds

with AnimationManager(n_cols=2, n_rows=1, figsize=(10, 5)) as am:
    for current_t in np.linspace(100, 110, 51):
        am.axs[0,0].cla()

        n_columns = np.floor(tail_length / d.bin_width).astype(int)
        idx = np.nonzero(~(d.neural_data.t < current_t))[0][0]
        
        
        am.axs[0,0].imshow(d.neural_data.a[idx-n_columns:idx,0,:].T, aspect='auto', interpolation='none', extent=[current_t - tail_length, current_t, d.neural_data.a.shape[2], 0])



        old_lims = am.axs[0,1].axis()
        am.axs[0,1].cla()
        am.axs[0,1].axis('off')
        
        s = ((current_t - tail_length) < d.behavioral_data.t) & (d.behavioral_data.t < current_t)
        am.axs[0,1].plot(d.behavioral_data.a[s,0,0], d.behavioral_data.a[s,0,1])
        pf.use_bigger_lims(am.axs[0,1], old_lims)
        
        am.grab_frame()